In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/Mestrado/2021.2/Aprendizagem de Maquina/trabalho_04/')

Mounted at /content/drive


# Imports

In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import math
import scipy
from mpl_toolkits import mplot3d
from matplotlib.collections import PolyCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors as mcolors
from IPython.display import display

import matplotlib.pyplot as plt

In [3]:
from src.utils import *

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [118]:
def transform(x):
    # if x == 'setosa':
    #     return 0
    # elif (x == 'virginica') or (x == 'versicolor'):
    #     return 1
    if (x == 'setosa') or (x == 'virginica'):
        return 0
    elif (x == 'versicolor'):
        return 1

In [119]:
df_iris = sns.load_dataset('iris')
df_iris['target'] = df_iris['species'].apply(lambda x: transform(x))
df_iris.drop(['species'], axis=1, inplace=True)

In [120]:
df_iris.head(3)

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [121]:
df_iris['target'].value_counts()

0    100
1     50
Name: target, dtype: int64

# Classificador

In [130]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import scipy
from mpl_toolkits import mplot3d
from matplotlib.collections import PolyCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors as mcolors
import json

import matplotlib.pyplot as plt

class NormalMultivariate():

    def __init__(self, df):
        self.df = df
        self.dim = len(df.columns)
        self.mean = np.array([df[col].mean() for col in df.columns  if col != 'target'])
        self.cov = np.matrix(df.cov().values)
        self.det = np.linalg.det(self.cov)
        self.peak_distribution = 1 / (((2 * np.pi) ** (self.dim / 2)) * 
                                      (self.det ** 0.5))
        self.pseudo_peak_distribution = None
    
    def __compute_pseudo_metrics(self):
        self.pseudo_inv = np.linalg.pinv(self.cov)
        eig_values = np.linalg.eig(self.cov)
        self.pseudo_determinent = np.product([i for i in eig_values[0] if i > 1e-12])
        self.pseudo_peak_distribution = 1 / (((2 * np.pi) ** (self.dim / 2)) * 
                                             (self.pseudo_determinent ** 0.5))

    def pdf(self, x):
        if isinstance(x, pd.core.frame.DataFrame) or isinstance(x[0], list) or isinstance(x[0], np.ndarray) or isinstance(x[0], np.matrix):
            if isinstance(x, pd.core.frame.DataFrame):
                x = x.values
            list_return = []
            for i in x:
                list_return.append(self.__pdf_single_list(i))
            return list_return

        else:
            return self.__pdf_single_list(x)

    def __pdf_single_list(self, x):

        if not isinstance(x, np.ndarray):
            x = np.array(x)
        x_mu = np.matrix(x - self.mean)
        try:
            inv = self.cov.I
            return self.peak_distribution * math.pow(math.e, -0.5 * (x_mu * inv * x_mu.T))
        except np.linalg.LinAlgError as err:
            if self.pseudo_peak_distribution is None:
                self.__compute_pseudo_metrics()
            return self.pseudo_peak_distribution * math.pow(math.e, -0.5 * (x_mu * self.pseudo_inv * x_mu.T))
             
        


    def plot_pdf(self, col_x1, col_x2):
        
        if self.dim > 2:
            raise NameError('Viewing can only be done with 2 attributes')
        x = np.linspace(self.df[col_x1].min(), self.df[col_x1].max(), 100)
        y = np.linspace(self.df[col_x2].min(), self.df[col_x2].max(), 100)
        df = pd.DataFrame(list(zip(x,y)), columns=[col_x1, col_x2])
        z = self.pdf(df)
        df['prob'] = z

        threedee = plt.figure(figsize=(15, 12)).gca(projection='3d')
        threedee.plot(df[col_x1], df[col_x2], df['prob'])
        threedee.set_xlabel(col_x1)
        threedee.set_ylabel(col_x2)
        threedee.set_zlabel('prob')

        plt.title(f'MÃƒÂ©dia = {self.mean},  cov = {self.cov}')
        plt.show()

class ClassifierBayes():
      
    def fit(self, X_train, y_train):
        df_train = X_train.copy()
        df_train['target'] = y_train
        self.list_class = df_train['target'].unique()
        self.features = [col for col in df_train.columns if col != 'target']
    
        self.dict_priori = {}
        self.dict_df_class = {}
        self.dict_norm_class = {}
    
        for j in sorted(self.list_class):
            self.dict_priori[j] = (len(df_train.loc[df_train['target'] == j]) / len(df_train))
            self.dict_df_class[j] = df_train[df_train['target'] == j].copy()
            self.dict_norm_class[j] = NormalMultivariate(self.dict_df_class[j][self.features])   

    
    def predict(self, df):

        list_predict = []
        for idx, row in df.iterrows():
    
            x = df[self.features].iloc[idx].tolist()
    
            pdf_priori = []
            dict_pdf = {}
            for j in sorted(self.list_class):
                verossimilhanca = self.dict_norm_class[j].pdf(x)
                dict_pdf[j] = verossimilhanca
                pdf_priori.append(
                    dict_pdf[j] * self.dict_priori[j]
                )
    
            normalize = sum(pdf_priori)
            prob_class = {}
            prob_max = -1
            class_predict = -1
            for j in sorted(self.list_class): 
                prob_class[j] =  (dict_pdf[j] * self.dict_priori[j]) / normalize
                if prob_class[j] > prob_max:
                    prob_max = prob_class[j]
                    class_predict = j
    
            list_predict.append(class_predict)
        
        return list_predict

    def predict_proba(self, df):

        dict_pred_classes = {}
        for i in self.list_class:
            dict_pred_classes[f'proba_{i}'] = []
        qt_classes = len(self.list_class)
        for idx, row in df.iterrows():
    
            x = df[self.features].iloc[idx].tolist()
    
            pdf_priori = []
            dict_pdf = {}
            for j in sorted(self.list_class):
                verossimilhanca = self.dict_norm_class[j].pdf(x)
                dict_pdf[j] = verossimilhanca
                pdf_priori.append(
                    dict_pdf[j] * self.dict_priori[j]
                )
    
            normalize = sum(pdf_priori)
            for j in sorted(self.list_class): 
                proba =  round((dict_pdf[j] * self.dict_priori[j]) / normalize, 2)
                dict_pred_classes[f'proba_{j}'].append(proba)

        df_proba = pd.DataFrame.from_dict(dict_pred_classes)
        return df.join(df_proba)


class Rejection():
    def __init__(self, rejection_cost, model='bayes', type_model='binary', max_output=1, min_output=0, threshold_decision=0.5):
        self.wr = rejection_cost
        self.max_output = max_output
        self.min_output = min_output
        self.best_t = np.inf
        self.threshold_decision = threshold_decision
        if model == 'bayes':
            self.model = ClassifierBayes()
        if type_model != 'binary':
            raise ValueError('This implementation only works for binary problems.')

    def fit(self, X_train, y_train, list_threshold, verbose=False):
        if len(list(set(y_train))) > 2:
            raise ValueError('This implementation only works for binary problems.')
        else:
            self.model.fit(X_train, y_train)
            self.df_prob = self.model.predict_proba(X_train.copy())
            self.df_prob['label'] = y_train
            self.qt_classes = len(self.model.list_class)
            return self.__minimize_rejection(list_threshold, verbose=verbose)

    def __minimize_rejection(self, list_threshold, verbose=False):

        iterations = {}
        min_t = np.inf
        min_sum = np.inf
        for t in list_threshold:
            qt_reject = 0 
            erro = 0
            for idx, row in (self.df_prob.iterrows()):
                label = row['label']
                prob_class_0 = row[f'proba_0']
                if (prob_class_0 >= self.threshold_decision - t) and (prob_class_0 <= self.threshold_decision + t):
                    qt_reject += 1
                elif prob_class_0 < (self.threshold_decision - t): 
                    class_predict = 1
                    if class_predict != label: erro += 1
                elif prob_class_0 > (self.threshold_decision + t): 
                    class_predict = 0
                    if class_predict != label: erro += 1

            iterations[f't={t}'] = {
                'r(t)': qt_reject / len(self.df_prob),
                'e(t)': erro / (len(self.df_prob) - qt_reject)
            }
            iterations[f't={t}']['sum'] = iterations[f't={t}']['e(t)'] + (self.wr * iterations[f't={t}']['r(t)'])
            if iterations[f't={t}']['sum'] < min_sum:
                min_sum = iterations[f't={t}']['sum']
                min_t = t

        self.best_t = min_t
        if verbose: 
            print(json.dumps(iterations, indent=4))
            print(f'Best t: {self.best_t}')

    def predict(self, df):

        if self.best_t == np.inf:
            raise ValueError('Minimizing rejection did not happen correctly. Perhaps your problem is not binary.')
        else:
            df_prob = self.model.predict_proba(df.copy())
            list_predict = []
            for idx, row in df_prob.iterrows(): 
                prob_first_class = row[f'proba_{self.model.list_class[0]}']
                if (prob_first_class >= self.threshold_decision - self.best_t) and (prob_first_class <= self.threshold_decision + self.best_t):
                    class_predict = -9999 
                elif prob_first_class < (self.threshold_decision - self.best_t): 
                    class_predict = 1
                elif prob_first_class > (self.threshold_decision + self.best_t): 
                    class_predict = 0
        
                list_predict.append(class_predict)
            
            return list_predict

In [131]:
X_train, y_train, X_test, y_test = split_train_test(df_iris.copy(), col_target='target', train_size=0.8, stratify=True)

In [132]:
bayes = Rejection(rejection_cost=0.05, model='bayes', type_model='binary', max_output=1, min_output=0, threshold_decision=0.5)

bayes.fit(X_train, y_train, list_threshold=[0.05, 0.10, 0.15, 0.20, 0.25, 0.30], verbose=True)

{
    "t=5e-05": {
        "r(t)": 0.0,
        "e(t)": 0.03333333333333333,
        "sum": 0.03333333333333333
    },
    "t=0.003": {
        "r(t)": 0.0,
        "e(t)": 0.03333333333333333,
        "sum": 0.03333333333333333
    },
    "t=0.05": {
        "r(t)": 0.008333333333333333,
        "e(t)": 0.025210084033613446,
        "sum": 0.025626750700280112
    },
    "t=0.1": {
        "r(t)": 0.008333333333333333,
        "e(t)": 0.025210084033613446,
        "sum": 0.025626750700280112
    },
    "t=0.15": {
        "r(t)": 0.03333333333333333,
        "e(t)": 0.017241379310344827,
        "sum": 0.018908045977011494
    },
    "t=0.2": {
        "r(t)": 0.058333333333333334,
        "e(t)": 0.008849557522123894,
        "sum": 0.01176622418879056
    },
    "t=0.25": {
        "r(t)": 0.08333333333333333,
        "e(t)": 0.00909090909090909,
        "sum": 0.013257575757575756
    },
    "t=0.3": {
        "r(t)": 0.11666666666666667,
        "e(t)": 0.0,
        "sum": 0.00583

In [133]:
X_test['predict'] = bayes.predict(X_test.copy())

In [134]:
X_test['target'] = y_test

In [135]:
X_test['predict'].value_counts()

 0       17
-9999     7
 1        6
Name: predict, dtype: int64

In [136]:
X_test['target'].value_counts()

0    20
1    10
Name: target, dtype: int64

In [137]:
def accuracy(y_true, y_pred): # Taxa de acerto
    
    qtd_acertos = 0
    for true, pred in zip(y_true, y_pred):
        if true == pred: 
            qtd_acertos += 1

    return qtd_acertos / len(y_true)

In [141]:
X_test_without_reject = X_test[X_test['predict'] != -9999].copy()

In [142]:
X_test_without_reject['predict'].value_counts()

0    17
1     6
Name: predict, dtype: int64

In [143]:
accuracy(X_test_without_reject['target'], X_test_without_reject['predict'])

1.0